In [1]:
import polars as pl
import quak
import numpy as np
import bpy
import ipywidgets as widgets
from IPython.display import display

df = pl.read_csv("a_df.csv")
reference_frame = pl.read_csv("b_reference_frame.csv")

widget = quak.Widget(df)
widget

Widget(sql='SELECT * FROM "df"')

In [2]:
blend_file_path = "housing_data_igor.blend"
bpy.ops.wm.open_mainfile(filepath=blend_file_path)

{'FINISHED'}

In [3]:
vertices = [(row["longitude_normalized"], row["latitude_normalized"], 0) for row in reference_frame.iter_rows(named=True)]

mesh = bpy.data.meshes.new("NormalizedMesh")
obj = bpy.data.objects.new("CaliforninaNormalizedObject", mesh)
bpy.context.collection.objects.link(obj)

mesh.from_pydata(vertices, [], [])
mesh.update()
obj.modifiers.new(name="GeometryNodes", type='NODES').node_group = bpy.data.node_groups["geo_house"]
bpy.context.view_layer.objects.active = obj
obj.select_set(True)
reference_frame.head()

output = widgets.Output()

def on_data_change(change=None):
    output.clear_output(wait=True)
    with output:
        widget_df = widget.data().pl()

        plotting_frame = reference_frame.with_columns(
            pl.lit(0).alias("custom_plotting")
        ).join(
            widget_df.select(["short_id", "median_house_value"]), 
            on="short_id", 
            how="left"
        ).with_columns(
            pl.when(pl.col("median_house_value").is_not_null())
            .then(pl.col("median_house_value"))
            .otherwise(pl.col("custom_plotting"))
            .alias("custom_plotting")
        ).select(["short_id", "custom_plotting"])

        custom_plotting_list = plotting_frame["custom_plotting"].to_list()
        normalized_values = list(np.interp(custom_plotting_list, 
                                           (min(custom_plotting_list), max(custom_plotting_list)), 
                                           (0.1, 3)))

        obj = bpy.data.objects['CaliforninaNormalizedObject']
        attr_name = 'median_house_value'
        attr = obj.data.attributes.get(attr_name) or obj.data.attributes.new(
            name=attr_name,
            type='FLOAT',
            domain='POINT'
        )
        attr.data.foreach_set('value', normalized_values)
        obj.data.update()

widget.observe(on_data_change, names=["sql"])
on_data_change(change= None)